ヒストグラムの作成(hist_ini,hist_shock)<br>
方針<br>
・横軸は10分割した、Ca2+イメージング強度の幅<br>
・縦軸は分布量<br>
・(ini)FZとnonFZ、それぞれにおいてヒストグラムを作成(横並べ)<br>
・Fzの判断は、mconseq(元データのFZでなく、処理後のFZ)<br>
・(shock)before shockとafter shock、それぞれにおいてヒストグラムを作成(横並べ)<br>

In [ ]:
#import modules

#dataframe
import pandas as pd

#number processing
import numpy as np

#standard scaling
from sklearn.preprocessing import StandardScaler

#plotting
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

#heatmap
import seaborn as sns

#math
import math

In [7]:
#make the list

#mouse
mouse_list = ["ID138464-1","ID138464-2","ID138464-3","ID138464-4","ID138464-5","ID138464-6"]
#all phase 
phaseall_list=["HC1","B1","B2","HC2","A1","A2","A31","A32","A41","A42","HC3","A5","B3","A6","A7","HC4","B4"]
#phase targeted for anlysis
phase_list=["A1","A2","A31","A32","A41","A42","A5","A6","A7"]

hist_ini<br>
現在ver<br>

In [ ]:
#　分析実行
for m in range(0,len(mouse_list)):

    mouse = mouse_list[m]
    
    #神経データをデータフレームにインポートする・Accepted cellのみ抽出する
    inputf='%s/python/%s_Longitudinal_Traces.csv' %(mouse,mouse)
    dff=pd.read_csv(inputf,header=0)
    dff0=pd.DataFrame()
    dff0=dff.iloc[0,:]
    Accept=dff0.index[dff0==' accepted']
    dfa=pd.DataFrame()
    dfa['Time2']=dff.iloc[1:,0]
    for accept_c in range (0,len(Accept)):
        dfa[Accept[accept_c]]=dff[Accept[accept_c]].iloc[1:]
        dfa=dfa.astype('float')

    #Acceped tcellのfirst cellとlast cellを抽出
    cell_f=dfa.columns[1]
    cell_l=dfa.columns[dfa.shape[1]-1]
    
    for ph in range(0,len(phase_list)):
        
        phase = phase_list[ph]
        
        inputf='%s/python/%s_%s_mean.csv' %(mouse,mouse,phase)
        dfmean=pd.read_csv(inputf,header=0)
        
        #神経発火活動の時系列(スライス)変化図:sliceini(X:Time1,Y:Ca2+img強度」)を作成
        for j in range(dfmean.columns.get_loc(cell_f),dfmean.columns.get_loc(cell_l)+1):
            plt.figure()
            plt.figure(figsize=(5.4, 3.6), dpi=80)
            plt.plot(dfmean['Slice'],dfmean[dfmean.columns[j]],color='black')
            plt.ylim(-0.05,0.30)
            for i in range(0,dfmean.shape[0]):
                if dfmean.iloc[i].loc['FZ'] == 1:
                    if dfmean.iloc[i].loc['X']  <= 125:
                        if dfmean.iloc[i].loc['Y']  <=100:
                            Color='lightpink'
                        else:
                            Color='lightblue'
                    else:
                        if dfmean.iloc[i].loc['Y']  <=100:
                            Color='silver'
                        else:
                            Color='lightgreen'
                    plt.fill([dfmean.iloc[i].loc['Slice'],dfmean.iloc[i].loc['Slice']+1,dfmean.iloc[i].loc['Slice']+1,dfmean.iloc[i].loc['Slice']],[-0.05,-0.05,0.30,0.30],color=Color)
                if (dfmean.iloc[i].loc['Shock'] == 1):
                    plt.plot([dfmean.iloc[i].loc['Slice'],dfmean.iloc[i].loc['Slice']],[-0.05,0.30],color='red',linestyle='dashed')
            plt.xlabel('Slice')
            plt.ylabel('Ca2+Img')
            title_name='sliceini_%s_%s' %(phase,dfmean.columns[j])
            plt.title(title_name)
            file_name='%s/python/%s_fig/'%(mouse,mouse)+title_name+'.png'
            plt.savefig(file_name)
            plt.figure()

            #カルシウム2+イメージング強度分布ヒストグラム:histini(X:img強度,Y:分布)を作成(現在は不使用)
            for j in range(dfmean.columns.get_loc(cell_f),dfmean.columns.get_loc(cell_l)+1):   
                plt.figure(figsize=(5.4, 3.6), dpi=80)
                df_m=dfmean[(dfmean['mconseq_0']<0)]
                df_p=dfmean[(dfmean['mconseq_0']>=0) & (dfmean['valid_0']==1)]
                plt.hist([df_m[dfmean.columns[j]],dfmean[dfmean.columns[j]],df_p[dfmean.columns[j]]],bins=10,normed=True,label=['nonFZ','ALL','FZ'])
                plt.xlabel('Ca2+img')
                plt.ylabel('density')
                title_name='histini_%s_%s' % (phase,dfmean.columns[j])
                file_name='%s/python/%s_fig/'%(mouse,mouse)+title_name+'.png'
                plt.title(title_name)
                plt.savefig(file_name)
                plt.figure()

hist_ini<br>
過去ver<br>
現在は不使用<br>

In [8]:
plt.figure(figsize=(5.4, 3.6), dpi=80)

for ph in range(0,1):
    
    phase=phase_list[ph]

    #for index in range(0,len(Index)):

    inputXY='ID138464-%s_%s_XY.txt' %(mouse,phase)

    dfx = pd.read_table(inputXY,header=0)

    inputT='ID138464-%s_long_trace_%s.csv'%(mouse,phase)

    dfl=pd.read_csv(inputT,header=0)

    dfx_m=pd.DataFrame(columns=dfx.columns,index=np.arange(0,int(dfx.iloc[dfx.shape[0]-1].loc['Slice']-1)*10))

    for i in range(2,int(dfx.shape[0])):

        for j in range(0,10):

            for k in range(0,len(dfx_m.columns)):

                dfx_m.iloc[(i-2)*10+j].loc[dfx_m.columns[k]]=dfx.iloc[i].loc[dfx_m.columns[k]]

    df=pd.concat([dfx_m, dfl], axis=1)
    df=df.dropna(how='any')
    
    

    for j in range(df.columns.get_loc(' C000'),df.columns.get_loc(' C000')+1):   

        df_nfz=df[(df['FZ']==0)]

        df_fz=df[(df['FZ']==1)]

        plt.hist([df_nfz[df.columns[j]],df_fz[df.columns[j]]],bins=10,normed=True,label=['nonFZ','FZ'])
        #plt.hist(df_m[df.columns[j]])
        plt.xlabel('Ca2+img')
        plt.ylabel('density')

        plt.ylim(0,5)
        title_name='fig5/histbin_phase%s_mouse%s_%s' % (phase,mouse,'C000')
        plt.title(title_name)
        file_name=title_name+'.png'
        
        plt.savefig(file_name)
        
        #splt.show()

hist_shock<br>
現在ver<br>

In [ ]:
#　分析実行
for m in range(0,len(mouse_list)):

    mouse = mouse_list[m]
    
    #神経データをデータフレームにインポートする・Accepted cellのみ抽出する
    inputf='%s/python/%s_Longitudinal_Traces.csv' %(mouse,mouse)
    dff=pd.read_csv(inputf,header=0)
    dff0=pd.DataFrame()
    dff0=dff.iloc[0,:]
    Accept=dff0.index[dff0==' accepted']
    dfa=pd.DataFrame()
    dfa['Time2']=dff.iloc[1:,0]
    for accept_c in range (0,len(Accept)):
        dfa[Accept[accept_c]]=dff[Accept[accept_c]].iloc[1:]
        dfa=dfa.astype('float')

    #Acceped tcellのfirst cellとlast cellを抽出
    cell_f=dfa.columns[1]
    cell_l=dfa.columns[dfa.shape[1]-1]
    
    for ph in range(0,len(phase_list)):
        
        phase = phase_list[ph]
        
        inputf='%s/python/%s_%s_mean.csv' %(mouse,mouse,phase)
        dfmean=pd.read_csv(inputf,header=0)
        
        if phase=='A2' or phase=='A6':

            #shockを与えるフェーズにおいて、mouseがshockを与えられた前と後での
            #カルシウム2+イメージング強度分布ヒストグラム:histini(X:img強度,Y:分布)を作成(現在は不使用)
            shocked_time=df_shock.iloc[0].loc['Slice']
            for j in range(dfmean.columns.get_loc(cell_f),dfmean.columns.get_loc(cell_l)+1):   
                df_b=dfmean[(dfmean['Slice']<shocked_time)]
                df_a=dfmean[(dfmean['Slice']>=shocked_time)]
                plt.figure()
                plt.hist([df_b[dfmean.columns[j]],df_a[dfmean.columns[j]]],bins=10,normed=True,label=['beforeSHOCK','afterSHOCK'])
                plt.xlabel('Ca2+img')
                plt.ylabel('density')
                #plt.ylim(0,5)
                title_name='histshock_%s_%s' % (phase,dfmean.columns[j])
                plt.title(title_name)
                file_name='%s/python/%s_fig/'%(mouse,mouse)+title_name+'.png'
                plt.savefig(file_name)
                plt.figure()

hist_shock<br>
過去ver<br>
現在は不使用<br>

In [ ]:
"""
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt

phase_list=["A2","A6"]

mouse=6

plt.figure(figsize=(5.4, 3.6), dpi=80)

for ph in range(0,1):
    
    phase=phase_list[ph]

    #for index in range(0,len(Index)):

    inputXY='ID138464-%s_%s_XY.txt' %(mouse,phase)

    dfx = pd.read_table(inputXY,header=0)

    inputT='ID138464-%s_long_trace_%s.csv'%(mouse,phase)

    dfl=pd.read_csv(inputT,header=0)
    
    dfx_m=pd.DataFrame(columns=dfx.columns,index=np.arange(0,int(dfx.iloc[dfx.shape[0]-1].loc['Slice']-1)*10))

    for i in range(2,int(dfx.shape[0])):

        for j in range(0,10):

            for k in range(0,len(dfx_m.columns)):

                dfx_m.iloc[(i-2)*10+j].loc[dfx_m.columns[k]]=dfx.iloc[i].loc[dfx_m.columns[k]]

    
    df=pd.concat([dfx_m, dfl], axis=1)
    df=df.dropna(how='any')
    
    

    
    df_shock=df[df['Shock']==1]
    
    shocked_time=df_shock.iloc[0].loc['Time1']

    for j in range(df.columns.get_loc(' C000'),df.columns.get_loc(' C000')+1):   

        df_b=df[(df['Time1']<shocked_time)]

        df_a=df[(df['Time1']>=shocked_time)]

        plt.hist([df_b[df.columns[j]],df_a[df.columns[j]]],bins=10,normed=True,label=['beforeSHOCK','afterSHOCK'])
        #plt.hist(df_m[df.columns[j]])
        plt.xlabel('Ca2+img')
        plt.ylabel('density')

        
        plt.ylim(0,5)
        title_name='fig4/histshock_phase%s_mouse%s_%s' % (phase,mouse,'C000')
        plt.title(title_name)
        file_name=title_name+'.png'
        plt.savefig(file_name)
        
        #splt.show()
        
"""